In [1]:
#cp ../../../downloads/roster_2023.parquet notebooks/roster_2023.parquet
#cp ../../../downloads/roster_2022.parquet notebooks/roster_2022.parquet
#cp ../../../downloads/roster_2021.parquet notebooks/roster_2021.parquet
#cp ../../../downloads/roster_2020.parquet notebooks/roster_2020.parquet
#cp ../../../downloads/roster_2019.parquet notebooks/roster_2019.parquet
#cp ../../../downloads/roster_2018.parquet notebooks/roster_2018.parquet
#cp ../../../downloads/player_stats_2017.parquet notebooks/player_stats_2017.parquet
#cp ../../../downloads/player_stats_2018.parquet notebooks/player_stats_2018.parquet
#cp ../../../downloads/player_stats_2019.parquet notebooks/player_stats_2019.parquet
#cp ../../../downloads/player_stats_2020.parquet notebooks/player_stats_2020.parquet
#cp ../../../downloads/player_stats_2021.parquet notebooks/player_stats_2021.parquet
#cp ../../../downloads/player_stats_2022.parquet notebooks/player_stats_2022.parquet
#cp ../../../downloads/"games.csv" notebooks/games.csv

In [2]:
import pandas as pd
import numpy as np
import os

games = pd.read_csv('../data/games.csv')
pd.set_option("display.max_columns",55)

games['surface']=games['surface'].str.replace(" ","")
games['roof']=games['roof'].str.replace(" ","")
away_surface_roof = games.groupby(['season','home_team']).agg({
    'surface':'first',
    'roof':'first'
}).reset_index()
away_surface_roof.columns = ['season','away_team','away_surface','away_roof']
games = games.merge(away_surface_roof,on=['season','away_team'],how='left')

roof = pd.get_dummies(games['roof'],drop_first=True,dtype=int,prefix='roof')
surface = pd.get_dummies(games['surface'],drop_first=True,dtype=int,prefix='surface')
away_roof = pd.get_dummies(games['away_roof'],drop_first=True,dtype=int,prefix='away_roof')
away_surface = pd.get_dummies(games['away_surface'],drop_first=True,dtype=int,prefix='away_surface')


games[roof.columns]=roof
games[surface.columns]=surface
games[away_roof.columns]=away_roof
games[away_surface.columns]=away_surface

games = games[games['season']>=2018].copy()


In [3]:
all_data = os.listdir("../data/")
roster_list = [i for i in all_data if 'roster_' in i]
stats_list = [i for i in all_data if 'player_stats_' in i]

all_rosters = pd.DataFrame()
all_stats = pd.DataFrame()

for filename in roster_list:
    tmp = pd.read_parquet(f"../data/{filename}")
    all_rosters = pd.concat([all_rosters,tmp])

for filename in stats_list:
    tmp = pd.read_parquet(f"../data/{filename}")
    all_stats = pd.concat([all_stats,tmp])

all_stats['merge_season'] = all_stats['season']+1

all_rosters['player_display_name'] = all_rosters['full_name'] 
all_rosters['merge_season'] = all_rosters['season']

agg_stats = all_stats[all_stats['season_type']=='REG'].groupby(
    ['player_display_name','merge_season']).agg({
        'completions':'mean',
        'attempts':'mean',
        'passing_yards':'mean',
        'passing_tds':'mean',
        'interceptions':'mean',
        'sacks':'mean',
        'sack_yards':'mean',
        'sack_fumbles':'mean',
        'sack_fumbles_lost':'mean',
        'passing_air_yards':'mean',
        'passing_yards_after_catch':'mean',
        'passing_first_downs':'mean',
        'passing_epa':'mean',
        'passing_2pt_conversions':'mean',
        'pacr':'mean',
        'carries':'mean',
        'rushing_yards':'mean',
        'rushing_tds':'mean',
        'rushing_fumbles':'mean',
        'rushing_fumbles_lost':'mean',
        'rushing_first_downs':'mean',
        'rushing_epa':'mean',
        'rushing_2pt_conversions':'mean',
        'receptions':'mean',
        'targets':'mean',
        'receiving_yards':'mean',
        'receiving_tds':'mean',
        'receiving_fumbles':'mean',
        'receiving_fumbles_lost':'mean',
        'receiving_air_yards':'mean',
        'receiving_yards_after_catch':'mean',
        'receiving_first_downs':'mean',
        'receiving_epa':'mean',
        'receiving_2pt_conversions':'mean',
        'racr':'mean',
        'target_share':'mean',
        'air_yards_share':'mean',
        'wopr':'mean',
        'special_teams_tds':'mean',
        'fantasy_points':'mean',
        'fantasy_points_ppr':'mean',
        'week':'count'
    }).reset_index()

agg_stats.rename({'week':'games'},axis=1,inplace=True)
agg_stats = pd.DataFrame(np.where(agg_stats==0,np.nan,agg_stats),columns=agg_stats.columns)

roster_stats = all_rosters.merge(agg_stats,on=['player_display_name','merge_season'],how='inner')

team_stats = roster_stats.groupby(['team','season']).agg({
        i:'mean' for i in agg_stats.columns if i not in ['player_display_name','merge_season']
    }).reset_index()

team_stats.to_csv("../data/seasonal_team_stats.csv",index=False)
team_stats

,team,season,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,games
0,ARI,2018,13.104167,18.854167,133.520833,1.25,1.25,2.25,19.75,0.75,0.5,131.875,52.5625,6.354167,0.378922,NaN,1.022075,1.909076,7.434489,0.285714,0.75,0.625,0.444318,-1.275358,NaN,2.085106,3.288585,23.569911,0.223214,0.375,0.0625,26.160021,9.93882,1.14428,-0.059985,0.0625,0.958412,0.094695,0.086276,0.202436,0.090909,4.095876,5.833464,7.631579
1,ARI,2019,0.0625,0.0625,2.0,0.0625,NaN,NaN,NaN,NaN,NaN,1.75,0.25,0.0625,3.088043,NaN,1.142857,5.863298,24.407411,0.284578,0.12358,0.092803,1.321338,-0.558417,0.0625,2.18817,3.113727,23.603709,0.213666,0.098755,0.090909,27.641733,10.179338,1.184324,-0.106718,0.0625,0.646193,0.109787,0.09871,0.233777,1.0,4.825361,6.74001,8.611111
2,ARI,2020,6.136458,9.66875,68.839583,0.658333,0.408333,1.833333,11.489583,0.125,NaN,79.509375,42.369444,3.136458,0.111067,0.1875,0.754137,3.466962,18.33054,0.407143,0.165179,0.098214,1.29285,0.220788,NaN,2.126651,3.467086,23.194021,0.205214,0.095833,0.095833,27.874765,11.033123,1.267078,0.365638,0.078755,0.736288,0.107211,0.108026,0.236434,NaN,4.421025,6.228679,8.7
3,ARI,2021,8.2875,13.075,93.0875,0.71875,0.5,1.3125,7.333333,0.333333,0.25,106.05,45.825,4.4125,-1.723019,0.25,1.045422,3.542127,17.638037,0.403846,0.176282,NaN,1.561028,0.536516,NaN,2.421919,3.507688,27.156002,0.299297,0.125,0.125,29.079144,12.02452,1.358555,0.537222,0.090909,0.93291,0.100609,0.10214,0.222412,NaN,5.371639,7.066982,7.714286
4,ARI,2022,11.785714,16.5,133.071429,1.071429,0.428571,1.357143,10.714286,0.547619,0.142857,121.452381,65.738095,6.285714,2.779393,NaN,1.30535,3.166067,13.958537,0.39916,0.426667,1.0,1.075633,-0.051651,NaN,2.175102,3.05624,24.053247,0.25279,0.129464,0.0625,24.181456,11.886568,1.224075,0.536118,NaN,0.184231,0.095186,0.093098,0.207948,NaN,4.773145,6.389366,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,WAS,2019,13.911458,22.10625,148.03125,1.041667,0.8125,2.108333,12.929167,0.377778,0.08125,176.182292,86.673611,7.113542,-0.240369,NaN,1.025359,3.805719,17.809987,0.223317,0.157885,0.0625,1.50844,-0.044189,0.076923,1.955089,2.600983,18.007718,0.201219,0.072115,0.072115,17.401822,9.136338,1.141015,0.251389,0.076923,0.773673,0.08521,0.065375,0.170526,NaN,4.825996,6.346621,6.473684
188,WAS,2020,12.202279,15.081197,136.017094,1.042735,1.004274,3.380342,26.602564,0.551282,0.34188,125.792735,64.982906,7.253561,-1.832857,0.111111,1.202165,3.728426,16.67053,0.199725,0.208868,0.069712,0.914498,0.007208,0.0625,1.66456,2.598365,19.401815,0.182921,0.104762,0.142857,23.765417,8.878408,0.972809,0.371379,NaN,0.541121,0.091719,0.095611,0.204506,0.076923,4.433176,5.80399,9.294118
189,WAS,2021,13.679167,18.095013,153.316667,1.111111,0.712963,1.289683,12.055556,0.236111,0.25,160.908249,69.25,7.068056,0.73458,0.222222,0.923499,2.407607,12.977208,0.290754,0.53125,1.0,1.099513,0.229996,0.111111,2.628793,3.582776,26.567227,0.209756,0.100298,0.090675,22.241168,13.78757,1.528589,0.208513,NaN,0.176768,0.107147,0.092608,0.225546,NaN,6.37114,8.561801,7.842105
190,WAS,2022,16.001225,27.07598,164.425245,1.05719,0.783088,2.085784,13.909314,0.383987,0.231209,204.223448,79.790441,7.825572,-4.206813,0.117647,0.785577,2.978317,13.159572,0.180583,0.201681,0.123162,1.387123,0.113467,0.0625,1.73612,2.444634,17.929219,0.250621,0.116

In [4]:

prep_frame = games[['season','week','away_team','home_team','away_score','home_score','result','spread_line']+
                    [i for i in games.columns if ('roof_' in i)|('surface_' in i)|('away_team_id' in i)|('home_team_id' in i)]].copy()

away_team_stats = team_stats.copy()
home_team_stats = team_stats.copy()

away_team_stats.columns = ['away_'+i for i in team_stats.columns]
home_team_stats.columns = ['home_'+i for i in team_stats.columns]

home_team_stats['season'] = home_team_stats['home_season']
away_team_stats['season'] = away_team_stats['away_season']

home_team_stats.drop(["home_season"],axis=1,inplace=True)
away_team_stats.drop(["away_season"],axis=1,inplace=True)

merge = prep_frame.merge(away_team_stats,on=['away_team','season'],how='left').merge(home_team_stats,on=['home_team','season'],how='left')



In [5]:
stat_cols = [i[5:] for i in away_team_stats.columns if i not in ('season','away_team')]

for col in stat_cols:
    merge[f"diff_{col}"]= merge[f"home_{col}"] - merge[f"away_{col}"]

In [6]:
away_result = merge[['season','week','away_team','result','away_score','home_score']].copy()
home_result = merge[['season','week','home_team','result','home_score','away_score']].copy()
away_result['result'] = -1*away_result['result']
away_result.columns = ['season','week','team','result','points_scored','points_allowed']
home_result.columns = ['season','week','team','result','points_scored','points_allowed']
away_result['h/a'] = 'away_team'
home_result['h/a'] = 'home_team'

results = pd.concat([away_result,home_result])

results.sort_values(by=['season','week'],ascending=[True,True],inplace=True)

results['result_not_nan'] = np.where(~results['result'].isna(),1,0)
results['points_scored_not_nan'] = np.where(~results['points_scored'].isna(),1,0)
results['points_allowed_not_nan'] = np.where(~results['points_allowed'].isna(),1,0)
results['result']=results['result'].fillna(0)
results['points_scored']=results['points_scored'].fillna(0)
results['points_allowed']=results['points_allowed'].fillna(0)

results['cumeresult']=results.groupby(['season','team'])['result'].cumsum( )
results['cumepoints_scored']=results.groupby(['season','team'])['points_scored'].cumsum( )
results['cumepoints_allowed']=results.groupby(['season','team'])['points_allowed'].cumsum( )
results['cumecount']=results.groupby(['season','team'])['result_not_nan'].cumsum()
results['cumemeanresult'] = results['cumeresult']/results['cumecount']
results['cumemeanpoints_scored'] = results['cumepoints_scored']/results['cumecount']
results['cumemeanpoints_allowed'] = results['cumepoints_allowed']/results['cumecount']

results['cumemeanresult_shift1'] = results.groupby(['season','team'])['cumemeanresult'].shift(1).fillna(0)
results['cumemeanpoints_scored_shift1'] = results.groupby(['season','team'])['cumemeanpoints_scored'].shift(1).fillna(0)
results['cumemeanpoints_allowed_shift1'] = results.groupby(['season','team'])['cumemeanpoints_allowed'].shift(1).fillna(0)

away_cume_result = results[results['h/a']=='away_team'][['season','week','team','cumemeanresult_shift1',
                                                         'cumemeanpoints_scored_shift1','cumemeanpoints_allowed_shift1']].copy()
home_cume_result = results[results['h/a']=='home_team'][['season','week','team','cumemeanresult_shift1',
                                                         'cumemeanpoints_scored_shift1','cumemeanpoints_allowed_shift1']].copy()

away_cume_result.columns = ['season','week','away_team','away_cumemeanresult_shift1','away_cumemeanpoints_scored_shift1','away_cumemeanpoints_allowed_shift1']
home_cume_result.columns = ['season','week','home_team','home_cumemeanresult_shift1','home_cumemeanpoints_scored_shift1','home_cumemeanpoints_allowed_shift1']

merge = merge.merge(home_cume_result,on=['season','week','home_team'],how='left').merge(
    away_cume_result,on=['season','week','away_team'],how='left'
)

In [7]:
merge['coverage']=merge['result'] - merge['spread_line']
merge['cumemeanresult_diff'] = merge['home_cumemeanresult_shift1'] - merge['away_cumemeanresult_shift1']
merge['cumemeanpoints_scored_diff'] = merge['home_cumemeanpoints_scored_shift1'] - merge['away_cumemeanpoints_scored_shift1']
merge['cumemeanpoints_allowed_diff'] = merge['home_cumemeanpoints_allowed_shift1'] - merge['away_cumemeanpoints_allowed_shift1']
merge['home_offense_power'] = merge['home_cumemeanpoints_scored_shift1'] - merge['away_cumemeanpoints_allowed_shift1']
merge['away_offense_power'] = merge['away_cumemeanpoints_scored_shift1'] - merge['home_cumemeanpoints_allowed_shift1']

In [8]:
"""metadat = [
    'season',
    'week',
    'away_team',
    'home_team',
    'away_score',
    'home_score',
    'result',
    'spread_line',
    'coverage',
]
features = [
    i for i in merge.columns if i not in metadat
]

means = merge.groupby(['season'])[features].mean()
sdevs = merge.groupby(['season'])[features].std()

sdevs_sum = sdevs.sum(axis=0)
features=list(sdevs_sum[sdevs_sum>0].index)

means.columns = [i+"_MEAN" for i in means.columns]
sdevs.columns = [i+"_SDEV" for i in sdevs.columns]

means = means.reset_index().copy()
sdevs = sdevs.reset_index().copy()

#f = features[0]
for f in features:
    tmp = merge[['season',f]]\
        .merge(means[['season',f+"_MEAN"]],on='season',how='left')\
        .merge(sdevs[['season',f+"_SDEV"]],on='season',how='left')
    tmp[f] = (tmp[f] - tmp[f+"_MEAN"])/tmp[f+"_SDEV"]

    merge[f] = tmp[f]"""

'metadat = [\n    \'season\',\n    \'week\',\n    \'away_team\',\n    \'home_team\',\n    \'away_score\',\n    \'home_score\',\n    \'result\',\n    \'spread_line\',\n    \'coverage\',\n]\nfeatures = [\n    i for i in merge.columns if i not in metadat\n]\n\nmeans = merge.groupby([\'season\'])[features].mean()\nsdevs = merge.groupby([\'season\'])[features].std()\n\nsdevs_sum = sdevs.sum(axis=0)\nfeatures=list(sdevs_sum[sdevs_sum>0].index)\n\nmeans.columns = [i+"_MEAN" for i in means.columns]\nsdevs.columns = [i+"_SDEV" for i in sdevs.columns]\n\nmeans = means.reset_index().copy()\nsdevs = sdevs.reset_index().copy()\n\n#f = features[0]\nfor f in features:\n    tmp = merge[[\'season\',f]]        .merge(means[[\'season\',f+"_MEAN"]],on=\'season\',how=\'left\')        .merge(sdevs[[\'season\',f+"_SDEV"]],on=\'season\',how=\'left\')\n    tmp[f] = (tmp[f] - tmp[f+"_MEAN"])/tmp[f+"_SDEV"]\n\n    merge[f] = tmp[f]'

In [9]:
merge.to_csv("../data/model_data.csv",index=False)

In [10]:
merge

,season,week,away_team,home_team,away_score,home_score,result,spread_line,roof_dome,roof_open,roof_outdoors,surface_astroplay,surface_astroturf,surface_dessograss,surface_fieldturf,surface_grass,surface_matrixturf,surface_sportturf,away_roof_dome,away_roof_open,away_roof_outdoors,away_surface_astroplay,away_surface_astroturf,away_surface_dessograss,away_surface_fieldturf,away_surface_grass,away_surface_matrixturf,...,diff_receiving_fumbles,diff_receiving_fumbles_lost,diff_receiving_air_yards,diff_receiving_yards_after_catch,diff_receiving_first_downs,diff_receiving_epa,diff_receiving_2pt_conversions,diff_racr,diff_target_share,diff_air_yards_share,diff_wopr,diff_special_teams_tds,diff_fantasy_points,diff_fantasy_points_ppr,diff_games,home_cumemeanresult_shift1,home_cumemeanpoints_scored_shift1,home_cumemeanpoints_allowed_shift1,away_cumemeanresult_shift1,away_cumemeanpoints_scored_shift1,away_cumemeanpoints_allowed_shift1,coverage,cumemeanresult_diff,cumemeanpoints_scored_diff,cumemeanpoints_allowed_diff,home_offense_power,away_offense_power
0,2018,1,ATL,PHI,12.0,18.0,6.0,1.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,-0.022024,NaN,6.116102,-5.10033,-0.261316,-0.575954,NaN,-0.81889,0.021528,0.02568,0.050268,NaN,-1.777444,-1.994442,0.010033,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,2018,1,BUF,BAL,3.0,47.0,44.0,7.5,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,...,-0.002768,-0.009768,-9.456934,0.798904,-0.319149,-0.427868,NaN,0.347239,-0.018568,-0.03554,-0.05273,NaN,0.836502,0.8067,1.567935,0.0,0.0,0.0,0.0,0.0,0.0,36.5,0.0,0.0,0.0,0.0,0.0
2,2018,1,PIT,CLE,21.0,21.0,0.0,-3.5,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,...,-0.028869,NaN,7.413674,0.627792,-0.085822,-1.093834,NaN,-0.012276,0.0085,0.034709,0.037046,NaN,-1.047224,-0.588766,1.95,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
3,2018,1,CIN,IND,34.0,23.0,-11.0,-1.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,...,-0.005119,0.008036,-2.33056,-2.773277,-0.035823,-0.132227,NaN,-0.318107,0.004849,-0.00382,0.000674,NaN,-1.285164,-1.183756,-0.4875,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0
4,2018,1,TEN,MIA,20.0,27.0,7.0,-1.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,...,0.009425,0.000968,3.893336,1.975294,-0.022516,0.435889,NaN,-0.626291,0.011467,0.013188,0.026431,NaN,0.123814,0.240128,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,2023,18,ATL,NO,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,-0.020915,-0.043137,-1.475743,2.204226,0.399619,0.645938,-0.018301,0.66771,0.007877,-0.01187,0.003507,NaN,2.321614,2.777826,1.651316,4.0,19.2,15.2,-2.6,16.6,19.2,NaN,6.6,2.6,-4.0,0.0,1.4
1640,2023,18,PHI,NYG,NaN,NaN,NaN,NaN,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...,0.027634,0.01602,-2.094199,-3.71443,-0.158611,-0.47396,NaN,0.042609,-0.007805,0.008182,-0.005979,NaN,-1.930454,-1.849174,-2.32493,-18.2,12.4,30.6,7.4,28.2,20.8,NaN,-25.6,-15.8,9.8,-8.4,-2.4
1641,2023,18,LA,SF,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,...,-0.110948,-0.02406,-0.63888,1.79172,0.095736,0.449805,NaN,0.66677,-0.009754,0.001052,-0.020156,NaN,1.318332,0.79272,-0.141176,19.8,33.4,13.6,0.8,22.4,21.6,NaN,19.0,11.0,-8.0,11.8,8.8
1642,2023,18,JAX,TEN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,...,0.055964,0.027982,6.963106,-0.565565,-0.180921,-0.737727,0.094083,-1.295593,0.012955,0.038739,0.04655,NaN,-1.209926,-0.995319,-2.0,-1.0,17.6,18.6,0.6,21.0,20.4,NaN,-1.6,-3.4,-1.8,-2.8,2.4
